## Trying Whisper Without N-best outputs

**Install whisper from git**

Make sure visibility is public

In [1]:
!pip install git+https://github.com/johnlohjy/whisper.git@Nbest_transcriptions

  Cloning https://github.com/johnlohjy/whisper.git (to revision Nbest_transcriptions) to /tmp/pip-req-build-7fh4z7i2
  Running command git clone --filter=blob:none --quiet https://github.com/johnlohjy/whisper.git /tmp/pip-req-build-7fh4z7i2
  Running command git checkout -b Nbest_transcriptions --track origin/Nbest_transcriptions
  Switched to a new branch 'Nbest_transcriptions'
  Branch 'Nbest_transcriptions' set up to track remote branch 'Nbest_transcriptions' from 'origin'.
  Resolved https://github.com/johnlohjy/whisper.git to commit 7b1c525cb96e5d846973458fa4c4bda74bf79844
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


**Load the Datasets from HuggingFace**

In [2]:
!pip install datasets

In [3]:
#Login because of gated access
#Token is saved to cache and available across all huggingface_hub components
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# Load the dataset
# gigapseech takes about 5mins
from datasets import load_dataset
ds_train = load_dataset("speechcolab/gigaspeech", "xs")

In [5]:
# Seeing the dstrain dataset
ds_train

DatasetDict({
    train: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 9389
    })
    validation: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 6750
    })
    test: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 25619
    })
})

In [6]:
# Load the first example on the fly
ds_train["train"][0]

{'segment_id': 'YOU0000000315_S0000660',
 'speaker': 'N/A',
 'text': "AS THEY'RE LEAVING <COMMA> CAN KASH PULL ZAHRA ASIDE REALLY QUICKLY <QUESTIONMARK>",
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/77906a935e2abb2d9cf4f707fdb5c12b788afa9bc712c39fef49bf186c19af8f/xs_chunks_0000/YOU0000000315_S0000660.wav',
  'array': array([0.0005188 , 0.00085449, 0.00012207, ..., 0.00125122, 0.00076294,
         0.00036621]),
  'sampling_rate': 16000},
 'begin_time': 2941.889892578125,
 'end_time': 2945.070068359375,
 'audio_id': 'YOU0000000315',
 'title': 'Return to Vasselheim | Critical Role: VOX MACHINA | Episode 43',
 'url': 'https://www.youtube.com/watch?v=zr2n1fLVasU',
 'source': 2,
 'category': 24,
 'original_full_path': 'audio/youtube/P0004/YOU0000000315.opus'}

In [7]:
import whisper

model = whisper.load_model("base")
result = model.transcribe(ds_train["train"][0]["audio"]["path"], beam_size= 10)
print("")
print(result["text"])

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


UnboundLocalError: local variable 'seeks' referenced before assignment